In [1]:
!git clone https://github.com/jalan-subham/SINet

Cloning into 'SINet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 1), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 18.57 KiB | 9.29 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
!mv SINet/* .

In [4]:
!gdown --fuzzy https://drive.google.com/file/d/1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt/view

Downloading...
From (original): https://drive.google.com/uc?id=1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt
From (redirected): https://drive.google.com/uc?id=1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt&confirm=t&uuid=cdabbe3e-5e17-4ae5-90fa-069dd380602d
To: /content/TrainDataset.zip
100% 1.10G/1.10G [00:17<00:00, 61.3MB/s]


In [5]:
!unzip TrainDataset.zip -d Dataset

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/TrainDataset/GT/camourflage_00199.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00198.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00195.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00194.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00193.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00192.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00191.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00190.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00189.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00187.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00185.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00184.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00182.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00181.png  
  inflating: Dataset/TrainDataset/GT/camourflage_00180.png  
  inflating: Dataset/TrainDataset/

In [3]:
import torch
import argparse
from sinet import SINet_ResNet50
from dataloader import get_loader
from trainer import trainer, adjust_lr
import torch.nn as nn
# from apex import amp

In [9]:

if __name__ == "__main__":
    class Opt:
      def __init__(self):
        pass
    opt = Opt()
    opt.batchsize = 8
    opt.save_epoch = 10
    opt.save_model = './Snapshot/2020-CVPR-SINet/'
    opt.train_img_dir = './Dataset/TrainDataset/Image/'
    opt.train_gt_dir = './Dataset/TrainDataset/GT/'
    opt.epoch = 40
    opt.lr = 1e-4
    opt.trainsize = 352
    opt.clip = 0.5
    opt.decay_rate = 0.1
    opt.decay_epoch = 30
    # opt.gpu =

    #torch.cuda.set_device(0)

    model_SINet = SINet_ResNet50(channel=32).to("cuda")
    print('-' * 30, model_SINet, '-' * 30)

    optimizer = torch.optim.Adam(model_SINet.parameters(), opt.lr)
    LogitsBCE = torch.nn.BCEWithLogitsLoss()

    net = nn.DataParallel(model_SINet)

    train_loader = get_loader(opt.train_img_dir, opt.train_gt_dir, batchsize=opt.batchsize,
                              trainsize=opt.trainsize, num_workers=12)
    total_step = len(train_loader)

    print('-' * 30, "\n[Training Dataset INFO]\nimg_dir: {}\ngt_dir: {}\nLearning Rate: {}\nBatch Size: {}\n"
                    "Training Save: {}\ntotal_num: {}\n".format(opt.train_img_dir, opt.train_gt_dir, opt.lr,
                                                              opt.batchsize, opt.save_model, total_step), '-' * 30)

    for epoch_iter in range(1, opt.epoch):
        adjust_lr(optimizer, epoch_iter, opt.decay_rate, opt.decay_epoch)
        trainer(train_loader=train_loader, model=model_SINet,
                optimizer=optimizer, epoch=epoch_iter,
                opt=opt, loss_func=LogitsBCE, total_step=total_step)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] initialize weights from resnet50


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx